# Deploying ADK agent on Agent Engine from source files

This notebook provides a technical guide for deploying an AI agent built with the **Google Agent Development Kit (ADK)** to **Agent Engine** (Vertex AI Reasoning Engine) using local source files.

## Summary

The standard deployment method for Vertex AI Agent Engine often relies on "on-the-fly" serialization (using tools like `cloudpickle`) of existing Python objects. However, this process frequently fails when an agent object contains complex, non-serializable components such as active network connections, specific database drivers, or complex nested modules.

**The Solution:** This notebook demonstrates the **Source-Based Deployment** approach. By providing the path to your source code and specifying the entry point, Agent Engine builds the environment and instantiates the object directly in the cloud. This bypasses local serialization errors and ensures a more robust deployment for production-grade agents.

---

## Deployment Steps

The notebook follows a structured workflow to move your local ADK code to a managed cloud endpoint:

1. **Environment Initialization:** Sets up the Vertex AI SDK with your Project ID and target Location (e.g., `us-central1`).
2. **Source Preparation:**
* Creates a local directory (`sample_agent`) containing your logic.
* Defines `agent.py`, where the `AdkApp` and `LlmAgent` are instantiated.
* Defines `requirements.txt` to ensure the cloud environment has the correct versions of `google-adk` and related libraries.


3. **Client Configuration:** Initializes the `vertexai.Client` using `v1beta1` features required for Agent Engine management.
4. **Deployment Configuration:** Defines a configuration dictionary that maps the local source to the cloud:
* `source_packages`: The directory containing your code.
* `entrypoint_module`: The Python module to run (e.g., `sample_agent.agent`).
* `entrypoint_object`: The specific variable name of your `AdkApp` instance.


5. **Two-Phase Deployment (The "Resource Name" Cycle):**
* **Phase 1:** Creates a dummy deployment to generate a unique **Reasoning Engine Resource Name**.
* **Phase 2:** Updates the local source code with this resource name (required by `AdkApp`) and redeploys the finalized agent.


6. **Validation:** Implements a `ChatClient` to test the deployed agent via asynchronous streaming queries.

---

## Key Considerations

### 1. Avoiding Serialization Errors

By using `source_packages` and `entrypoint_module`, you are telling Vertex AI to "install and run" your code rather than "copy a snapshot of memory." This is the recommended path for any agent that uses complex third-party libraries or internal state that cannot be easily pickled.

### 2. The `app_name` Requirement

In the ADK framework, the `AdkApp` object requires its own GCP resource name (the `projects/.../reasoningEngines/...` string) to function correctly for session management. Because this name isn't known until *after* the first deployment, you must:

* Deploy with a placeholder.
* Retrieve the generated name.
* Update the source code and redeploy.

### 3. API Versioning

This notebook utilizes the `v1beta1` API version via `HttpOptions`. This is necessary for accessing the latest Agent Engine features within the Vertex AI SDK.

### 4. Dependency Management

Ensure that all libraries imported in your `agent.py` are explicitly listed in your `requirements.txt`. The cloud environment is built from scratch based on this file.


In [1]:
import vertexai
from google.genai.types import HttpOptions

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
%%bash
mkdir -p sample_agent
cat <<'EOF' >sample_agent/agent.py
from vertexai.agent_engines import AdkApp
from google.adk.agents import LlmAgent
from google.adk.sessions import VertexAiSessionService

root_agent = LlmAgent(
    name='frientdly_agent',
    model='gemini-2.5-flash',
    instruction='Be friendly.',
)

def session_service_builder():
    return VertexAiSessionService()

# entrypoint_object
app = AdkApp(
    agent=root_agent,
    # Need to be replaced by the full Reasoning Enigne resource name.
    app_name='__APP_NAME__',
    # Need to explicitly specify VertexAiSessionService.
    session_service_builder=session_service_builder,
)
EOF

cat <<'EOF' >sample_agent/requirements.txt
google-adk==1.21.0
google-cloud-aiplatform==1.132.0
google-genai==1.56.0
EOF

In [4]:
DISPLAY_NAME = 'frientdly_agent'

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1',
        base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

config = {
        'display_name': DISPLAY_NAME,
        'source_packages': ['sample_agent'],
        'entrypoint_module': 'sample_agent.agent',
        'entrypoint_object': 'app',
        'requirements_file': 'sample_agent/requirements.txt',
        'class_methods': [
            {
                'name': 'async_stream_query',
                'api_mode': 'async_stream',
                'description': 'Stream responses from the agent for a given query.',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'input': {'type': 'string', 'description': 'The user query'},
                        'config': {'type': 'object', 'description': 'Optional runtime config'}
                    },
                    'required': ['input']
                }
            },
            {
                'name': 'async_create_session',
                'api_mode': 'async',
                'description': 'Create a new managed session.'
            }
        ]
}

In [5]:
def get_resource_name(display_name):
    resource_name = None
    for item in client.agent_engines.list():
        if item.api_resource.display_name == display_name:
            resource_name = item.api_resource.name
            break
    return resource_name

if not get_resource_name(DISPLAY_NAME):
    # Deply dummy agent to assign Reasoning Engine resource name.
    remote_agent = client.agent_engines.create(
        config=config,
    )

# Set the resource name as app_name, and update the agent.
resource_name = get_resource_name(DISPLAY_NAME)
!sed -i 's:__APP_NAME__:{resource_name}:g' sample_agent/agent.py
remote_agent = client.agent_engines.update(
    name=resource_name,
    config=config,
)

In [6]:
remote_agent = client.agent_engines.get(name=resource_name)
remote_agent

AgentEngine(api_resource.name='projects/879055303739/locations/us-central1/reasoningEngines/6183817221844762624')

In [7]:
# Chat client to test AdkApp
class ChatClient:
    def __init__(self, app, user_id='default_user'):
        self._app = app
        self._user_id = user_id
        self._session_id = None

    async def async_stream_query(self, message):
        if not self._session_id:
            session = await self._app.async_create_session(
                user_id=self._user_id,
            )
            self._session_id = getattr(session, 'id', None) or session['id']

        result = []
        async for event in self._app.async_stream_query(
            user_id=self._user_id,
            session_id=self._session_id,
            message=message,
        ):
#            print('====')
#            print(event)
#            print('====')
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

In [9]:
client = ChatClient(remote_agent)
_ = await client.async_stream_query('how are you?')

Hello there! I'm doing very well, thank you for asking. I'm busy helping users like you and learning new things every day, which I find quite enjoyable.

How about you? How are you doing today? 😊
